In [17]:
file <- file.path(cmdstan_path(), "examples", "bernoulli", "bernoulli.stan")
mod <- cmdstan_model(file)

Warning message:
“Can't find CmdStan makefile to detect version number. Path may not point to valid installation.”


ERROR: Error in initialize(...): Assertion on 'stan_file' failed: File does not exist: '/Users/hyunjimoon/Dropbox/21S_paper/SBC-approx-sensitivity/sbc-approx/examples/bernoulli/bernoulli.stan'.


In [13]:
install_cmdstan(cores = 2, overwrite=TRUE)

The C++ toolchain required for CmdStan is setup properly!

* Latest CmdStan release is v2.27.0

* Installing CmdStan v2.27.0 in /Users/hyunjimoon/.cmdstan/cmdstan-2.27.0

* Downloading cmdstan-2.27.0.tar.gz from GitHub...

* Removing the existing installation of CmdStan...

* Download complete

* Unpacking archive...

* Building CmdStan binaries...



arch: posix_spawnp: make: Bad CPU type in executable



Warning message:
“There was a problem during installation. See the error message(s) above.”


In [4]:
library(SBC)
library(cmdstanr)
generator <- function(){
  function(){
    mu <- rnorm(1, 0, 5)
    tau <- rcauchy(1, 0, 5)
    theta_trans <- rnorm(8, 0, 1)
    theta <-theta_trans * tau + mu
    list(
      generated = rnorm(8, mu, sigma),
      parameters = list(
        mu = mu,
        theta_trans=theta_trans,
        tau = tau,
        theta = theta
      )
    )
  }
}


J <- 8
y <- c(28, 8, -3, 7, -1, 1, 18, 12)
sigma <- c(15, 10, 16, 11, 9, 11, 10, 18)

data = list("J"=J, "y"=y, "sigma"=sigma)
ncp_model = cmdstanr::cmdstan_model("models/eightschools_ncp/eightschools_ncp.stan")


workflow <- SBC::SBCWorkflow$new(ncp_model, generator())

workflow$simulate(10)
d <- workflow$fit_model(20, 20, data)
prior <- workflow$prior_samples
post <- workflow$posterior_samples
ranks <- workflow$calculate_rank()

Running MCMC with 1 chain...



Chain 1 dyld[35672]: Library not loaded: @rpath/libtbb.dylib

Chain 1   Referenced from: /Users/hyunjimoon/Dropbox/21S_paper/SBC-approx-sensitivity/sbc-approx/models/eightschools_ncp/eightschools_ncp

Chain 1   Reason: tried: '/Users/hyunjimoon/.cmdstanr/cmdstan-2.23.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file), '/Users/hyunjimoon/.cmdstanr/cmdstan-2.23.0/stan/lib/stan_math/lib/tbb/libtbb.dylib' (no such file), '/Library/Frameworks/R.framework/Resources/lib/libtbb.dylib' (no such file), '/Library/Java/JavaVirtualMachines/zulu-8.jdk/Contents/Home/jre/lib/server/libtbb.dylib' (no such file)

Warning message:
“Chain 1 finished unexpectedly!
”
Warning message:
“No chains finished successfully. Unable to retrieve the fit.”


ERROR: Error: No chains finished successfully. Unable to retrieve the draws.


In [6]:
set.seed(42)

In [1]:
library(cmdstanr)
#install_cmdstanr(overwrite = TRUE)
library(dplyr)
library(tidyverse)
library(reshape)
library(parallel)
library(posterior)
#library(rstanarm)
devtools::install_github("hyunjimoon/SBC",ref="workflow")
library(SBC)
set.seed(1954)
.libPaths("~/Rlib")
options(mc.cores = parallel::detectCores())
set_get_Dir <- function(modelName){
  scriptDir <- getwd()
  modDir <- file.path(scriptDir, "models")
  dataDir <- file.path(scriptDir, "data")
  delivDir <- file.path(scriptDir, "deliv", modelName)
  dir.create(delivDir)
  file <- file.path(modDir, modelName, paste0(modelName, ".stan"))
  mod <- cmdstan_model(file)
  return(list(data = data, mod = mod, modDir = modDir, file = file, delivDir = delivDir)) 
}

source("tools/selfCalib.R")
modelName = "simple.normal_1"
modDir <- set_get_Dir(modelName)$modDir
delivDir <- set_get_Dir(modelName)$delivDir
file <- set_get_Dir(modelName)$file
simple.normal_1 = cmdstanr::cmdstan_model(file)

This is cmdstanr version 0.4.0.9000

- Online documentation and vignettes at mc-stan.org/cmdstanr

- CmdStan path set to: /Users/hyunjimoon/.cmdstan/cmdstan-2.27.0

- Use set_cmdstan_path() to change the path


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ───────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ stringr 1.4.0
✔ tidyr   1.1.3     ✔ forcats 0.5.1
✔ readr   1.4.0     

── Conflicts ──────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape’


The following objects are masked from ‘package:tidyr’:

    expand, smiths


The following object is masked from ‘pack

In [2]:
generator <- 
  function(){
    theta <- rnorm(1, 0, 1)
    list(
      generated = rnorm(8, theta, 1),
      parameters = list(
        theta
      )
    )
  }


D <- 8
y <- c(1.7,1.6,1.9,1.3,3.5,3.1,1.1,3.6) #round(rnorm(10, 2, 1), 1)

data = list("D"=D, "y"=y, "theta_hp"=0)

nChains <- 4
parallel_chains <- min(nChains, detectCores())
N = 10 #200
M = 5 # 40
# data needed for y_sim placeholders
workflow <- SBC::SBCWorkflow$new(simple.normal_1, generator())

In [3]:
workflow$simulate(n_sbc_iterations = N) 
workflow$fit_model(sample_iterations = M, warmup_iterations = M, data)

ERROR: Error in do.call(generator, list(...)): 'what' must be a function or character string


In [10]:
SBCWorkflow <- R6::R6Class("SBCWorkflow",
  public = list(
    #' @field cmdstan_model A CmdStanModel object used for SBC
    #' @field brms_model_fit A brmsfit object used for SBC
    #' @field sim_function If using CmdStanModel, a simulator function, which should return simulated prior and data samples for SBC. Please refer \code{SBCWorkflow$initialize()} for details.
    #' @field calculated_ranks A named list of vectors which contained calculated ranks, or NULL if not calculated.
    #' @field prior_samples A posterior::draws_rvars of dimension(n_iterations=1, n_chains=n_sbc_iterations, n_variables=n_variables) which stores prior samples
    #' @field simulated_y A posterior::draws_rvars of dimension(n_iterations=n_simulated_y, n_chains=n_sbc_iterations, n_variables=1), which stores simulated y as "y"
    #' @field posterior_samples A posterior::draws_Rvars of dimension(n_iterations=n_posterior_samples, n_chains=n_sbc_iterations, n_variables=n_variables), which stores fitted posterior samples
    #' @field thin_factor Non-negative integer representing posterior thinning interval
    cmdstan_model = NULL,
    brms_model_fit = NULL,
    sim_function = NULL,
    calculated_ranks = NULL,
    prior_samples = NULL,  #  written on simulate
    simulated_y = NULL,  #  written on simulate
    posterior_samples = NULL, # written on fit_model
    thin_factor = NULL, # type is integer. written on initialize and fit_model

    #' R6 Initializer for SBCWorkflow
    #'
    #' @param model_obj A CmdStanModel or brmsfit object to use for fitting data.
    #' @param sim_function If using CmdStanModel, a simulator function. The function must return a
    #'   named list with elements \code{parameters} and \code{generated}.
    #'   Parameters should be a named list containing prior samplers for
    #'   parameters. Generated must be a 1 dimensional vector containing
    #'   simulated y data generated from the \code{parameters} samples.
    #'
    #' @return None
    initialize = function(model_obj, sim_function){
      if(!is.na(match(CMDSTAN_MODEL_CLASS_NAME, class(model_obj)))){
        if(missing(sim_function)){
          stop("If a cmdstan model is given, sim_function must also be supplied.")
        }
        self$cmdstan_model <- model_obj
        self$sim_function <- sim_function
      }
      else if(!is.na(match(BRMSFIT_MODEL_CLASS_NAME, class(model_obj)))){
        if(!missing(sim_function)){
          warning("A brm model has been supplied, but sim_function was also included in arguments. It will be ignored.")
        }
        self$brms_model_fit <- model_obj
      }
      else{
        stop(paste("The specified model object is neither a", CMDSTAN_MODEL_CLASS_NAME, "or a", BRMSFIT_MODEL_CLASS_NAME, ". Please recheck your model object."))
      }
    },

    #' Sample \eqn{\tilde{\theta} \sim P(\theta)} and \eqn{\tilde{y} \sim P(\tilde{y}) | \tilde{\theta} using \code{sim_function}
    #'
    #' @param n_sbc_iterations Number of prior sample sets to generate. Equates to number of SBC iterations
    #' @param ... Additional arguments to be passed to \code{sim_function}
    simulate = function(n_sbc_iterations, ...){  # number of simulation draws should be at least 1000
      if(!is.null(self$brms_model_fit)){
        #prior_sampler <- update(self$model_obj, sample_prior="only", chains=1, iter=n * 2, warmup = n)
      }
      else if(!is.null(self$cmdstan_model)){
        draws_rvars_list <- do.call(generator_to_draws_rvars, list(self$sim_function, n_sbc_iterations, ...))
        self$prior_samples <- draws_rvars_list[["parameters"]]
        self$simulated_y <- draws_rvars_list[["generated"]]
      }
      else{
        stop("No valid model specified for the workflow. Please check that your model is a valid brmsfit or a CmdStanModel object.")
      }
    },

    #' Sample \eqn{\widehat{\theta} \sim P(\widehat{\theta} | \tilde{y})} using the stan model.
    #'
    #' @param sample_iterations Number of sampling iterations. Equates to number of posterior samples
    #' @param warmup_iterations Number of warmup iteratioins.
    #' @param data List specifying data to be passed to the model. Note that \code{y} will be replaced with simulated \code{y} samples.
    #' @param thin_factor Non-negative integer in which thinning should be applied. 1 equates to no thinning being done.
    #'
    #' @return A list of named lists containing posterior samples for each parameter.
    fit_model = function(sample_iterations, warmup_iterations, data=list(), thin_factor=3){
      self$thin_factor <- thin_factor
      if(is.null(self$simulated_y)){
        stop("There are no simulated data available. Please run SBCWorkflow$simulate() first ")
      }
      # if(posterior::nchains(self$prior_samples) != posterior::nchains(self$simulated_y)){
      #   stop("The number of simulated prior and data do not match. Please rerun SBCWorkflow$simulate() and if using CmdStan, make sure your generator is well formed.")
      # }


      if(!is.null(self$brms_model_fit)){
        #brms
      }
      else if(!is.null(self$cmdstan_model)){
        sbc_iterations <- posterior::nchains(self$prior_samples)
        posterior_draws_rvars <- cmdstan_fits_to_draws_rvars(self$cmdstan_model, sbc_iterations, data, self$simulated_y, iter_sampling=sample_iterations * thin_factor, iter_warmup = warmup_iterations, chains=1, thin=thin_factor)
        self$posterior_samples <- posterior_draws_rvars
      }

      posterior_draws_rvars
    },

    #' Calculate rank statistics for a given parameter name
    #'
    #' @param param list of parameter names to calculate. If not given, calculate for all available parameters.
    #'
    calculate_rank = function(param=NULL){
      if(is.null(self$posterior_samples)){
        stop("There are no posterior samples available. Please run SBCWorkflow$fit_model() first.")
      }
      if(is.null(param)){
        param <- posterior::variables(prior)
      }
      calculate_rank_draws_rvars(self$prior_samples, self$posterior_samples)

      # ranks <- array(rep(0, n_iters * n_params), dim=c(n_iters, n_params))
      # dimnames(ranks)[2] <- list(names(decomposed_prior))
      #
      # for(i in 1:n_iters){
      #   prior <- self$prior_samples[[i]]
      #   decomposed_prior <- decompose_structure_to_par_list(prior)
      #
      #   for(j in 1:n_params){
      #     regex_res <- gregexpr("(?<=\\[)(.+)(?=\\])", param[[j]], perl = TRUE)[[1]]
      #     capture_start <- attr(regex_res, "capture.start")
      #     capture_length <- attr(regex_res, "capture.length")
      #     if(capture_start != -1){
      #       index <- as.integer(unlist(substr(param[[j]], capture_start, capture_start + capture_length - 1)))
      #       ranks[i, param[[j]]] <- sum(self$posterior_samples[[i]][[param[[j]]]] < access_element_by_index(self$prior_samples[[i]][[base_param_names[[j]]]], index))
      #     }
      #     else{
      #       t1 <- self$posterior_samples[[i]][[param[[j]]]]
      #       t2 <- self$prior_samples[[i]][[param[[j]]]]
      #       #ranks[i, param[[j]]] <- sum(self$posterior_samples[[i]][[param[[j]]]] < self$prior_samples[[i]][[param[[j]]]])
      #       ranks[i, param[[j]]] <- sum(t1 < t2)
      #     }
      #   }
      # }
      # return(ranks)
    },
    add_steps = function(N){

    }
  )
)

In [11]:
workflow$simulate(n_sbc_iterations = N) 
workflow$fit_model(sample_iterations = M, warmup_iterations = M, data)

ERROR: Error in do.call(generator_to_draws_rvars, list(self$sim_function, n_sbc_iterations, : object 'N' not found


In [12]:
workflow$simulate(n_sbc_iterations = N, custom_prior = NULL) 
workflow$fit_model(sample_iterations = M, warmup_iterations = M, data)

ERROR: Error in do.call(generator_to_draws_rvars, list(self$sim_function, n_sbc_iterations, : object 'N' not found


In [13]:
generator <- 
  function(){
    theta <- rnorm(1, 0, 1)
    list(
      generated = rnorm(8, theta, 1),
      parameters = list(
        theta
      )
    )
  }


D <- 8
y <- c(1.7,1.6,1.9,1.3,3.5,3.1,1.1,3.6) #round(rnorm(10, 2, 1), 1)

data = list("D"=D, "y"=y, "theta_hp"=0)

nChains <- 4
parallel_chains <- min(nChains, detectCores())
N = 10 #200
M = 5 # 40
# data needed for y_sim placeholders
workflow <- SBCWorkflow$new(simple.normal_1, generator())
workflow$simulate(n_sbc_iterations = N, custom_prior = NULL) 
workflow$fit_model(sample_iterations = M, warmup_iterations = M, data)

ERROR: Error in detectCores(): could not find function "detectCores"


In [14]:
next_workflow <- SBCWorkflow$new(simple.normal_1, next_generator())

ERROR: Error in match(CMDSTAN_MODEL_CLASS_NAME, class(model_obj)): object 'CMDSTAN_MODEL_CLASS_NAME' not found


In [110]:
next_workflow$simulate(n_sbc_iterations = N, theta_hp = summarise_draws(workflow$prior_samples)$mean) 

ERROR: Error in rnorm(1, theta_hp, 1): argument "theta_hp" is missing, with no default


In [103]:
next_generator <-  function(theta_hp){
    theta <- rnorm(1, theta_hp, 1)
    list(
      generated = rnorm(8, theta, 1),
      parameters = list(
        theta
      )
    )
  }

next_workflow <- SBCWorkflow$new(simple.normal_1, next_generator())
next_workflow$simulate(n_sbc_iterations = N, theta_hp = summarise_draws(workflow$prior_samples)$mean) 
next_workflow$fit_model(sample_iterations = M, warmup_iterations = M, data)

ERROR: Error in rnorm(1, theta_hp, 1): argument "theta_hp" is missing, with no default


[1] -0.1845643

In [ ]:
  t_prior <- prior[pars]
  workflow$simulate(n_sbc_iterations = N, custom_prior = prior)
  #clamped stan could be applied for inference
  D <- dim(as_draws_array(workflow$simulated_y))[1]
  y <- workflow$simulated_y
  prior_summ <- summarise_draws(workflow$prior_samples)
  theta_hp <- prior_summ$mean
  data = list("D"=D, "y"=y, "theta_hp"=theta_hp)
    

In [85]:
workflow$simulate(n_sbc_iterations = N, custom_prior = NULL)
workflow$simulated_y

# A draws_rvars: 8 iterations, 10 chains, and 1 variables
$y: rvar<8,10>[1] mean ± sd:
[1] 0.0023 ± 1.1 


In [86]:
workflow$prior_samples

# A draws_rvars: 1 iterations, 10 chains, and 1 variables
$...1: rvar<1,10>[1] mean ± sd:
[1] 0.16 ± 0.78 


In [89]:
workflow$simulate(n_sbc_iterations = N, custom_prior = prior)
workflow$simulated_y

# A draws_rvars: 8 iterations, 10 chains, and 1 variables
$y: rvar<8,10>[1] mean ± sd:
[1] 0.0035 ± 1.4 


In [90]:
workflow$prior_samples

# A draws_rvars: 1 iterations, 10 chains, and 1 variables
$...1: rvar<1,10>[1] mean ± sd:
[1] -0.18 ± 0.91 


In [49]:
pars = "theta"
t_prior <- prior[pars]
workflow$simulate(n_sbc_iterations = N, custom_prior = prior) #should recive both prior sample and prior distribution
#clamped stan could be applied for inference
D <- dim(as_draws_array(workflow$simulated_y))[1]
y <- workflow$simulated_y
prior_summ <- summarise_draws(workflow$prior_samples)
theta_hp <- prior_summ$mean
data = list("D"=D, "y"=y, "theta_hp"=theta_hp)
    

ERROR: Error in (function (D) : unused argument (custom_prior = list(list()))


In [93]:
prior <- workflow$prior_samples
post <- workflow$posterior_samples
ranks <- workflow$calculate_rank()
plot_ecdf(as_draws_matrix(ranks), "theta")
plot_hist(as_draws_matrix(ranks), "theta", bins = 5)
print_summary(as_draws_matrix(ranks), "theta", bins = 5)

ERROR: Error: The following variables are missing in the draws object: {'...1'}


In [ ]:
pars<-  "mu" #names(prior)[c(1)] 
evolve_df <- initDf(1, summary = "pars")
sc_workflow <- SBCWorkflow$new(ncp_model, generator())
sc_prior <- selfCalib(sc_workflow, prior, data, pars, N, M, cnt = 1, evolve_df, delivDir)[1]

# test self-consistency fo sc_prior
sc_workflow$simulate(n_sbc_iterations = N, custom_prior = sc_prior)
#clamped stan could be applied for inference
sc_workflow$fit_model(sample_iterations = M, warmup_iterations = M, data)
sc_ranks <- sc_workflow$calculate_rank()
plot_ecdf(as_draws_matrix(sc_ranks), "mu")
plot_hist(as_draws_matrix(sc_ranks), "mu", bins = 10)
print_summary(as_draws_matrix(sc_ranks), "mu", bins = 10)

In [92]:
#workflow.R
CMDSTAN_MODEL_CLASS_NAME <- "CmdStanModel"
BRMSFIT_MODEL_CLASS_NAME <- "brmsfit"

#' R6 class representing a fluid SBC workflow, boasting a highly customizable pipeline and supports intermediate updates
#' @export
SBCWorkflow <- R6::R6Class("SBCWorkflow",
                           public = list(
                             #' @field cmdstan_model A CmdStanModel object used for SBC
                             #' @field brms_model_fit A brmsfit object used for SBC
                             #' @field sim_function If using CmdStanModel, a simulator function, which should return simulated prior and data samples for SBC. Please refer \code{SBCWorkflow$initialize()} for details.
                             #' @field calculated_ranks A named list of vectors which contained calculated ranks, or NULL if not calculated.
                             #' @field prior_samples A posterior::draws_rvars of dimension(n_iterations=1, n_chains=n_sbc_iterations, n_variables=n_variables) which stores prior samples
                             #' @field simulated_y A posterior::draws_rvars of dimension(n_iterations=n_simulated_y, n_chains=n_sbc_iterations, n_variables=1), which stores simulated y as "y"
                             #' @field posterior_samples A posterior::draws_Rvars of dimension(n_iterations=n_posterior_samples, n_chains=n_sbc_iterations, n_variables=n_variables), which stores fitted posterior samples
                             #' @field thin_factor Non-negative integer representing posterior thinning interval
                             cmdstan_model = NULL,
                             brms_model_fit = NULL,
                             sim_function = NULL,
                             calculated_ranks = NULL,
                             prior_samples = NULL,  #  written on simulate
                             simulated_y = NULL,  #  written on simulate
                             posterior_samples = NULL, # written on fit_model
                             thin_factor = NULL, # type is integer. written on initialize and fit_model
                             
                             #' R6 Initializer for SBCWorkflow
                             #'
                             #' @param model_obj A CmdStanModel or brmsfit object to use for fitting data.
                             #' @param sim_function If using CmdStanModel, a simulator function. The function must return a
                             #'   named list with elements \code{parameters} and \code{generated}.
                             #'   Parameters should be a named list containing prior samplers for
                             #'   parameters. Generated must be a 1 dimensional vector containing
                             #'   simulated y data generated from the \code{parameters} samples.
                             #'
                             #' @return None
                             initialize = function(model_obj, sim_function){
                               if(!is.na(match(CMDSTAN_MODEL_CLASS_NAME, class(model_obj)))){
                                 if(missing(sim_function)){
                                   stop("If a cmdstan model is given, sim_function must also be supplied.")
                                 }
                                 self$cmdstan_model <- model_obj
                                 self$sim_function <- sim_function
                               }
                               else if(!is.na(match(BRMSFIT_MODEL_CLASS_NAME, class(model_obj)))){
                                 if(!missing(sim_function)){
                                   warning("A brm model has been supplied, but sim_function was also included in arguments. It will be ignored.")
                                 }
                                 self$brms_model_fit <- model_obj
                               }
                               else{
                                 stop(paste("The specified model object is neither a", CMDSTAN_MODEL_CLASS_NAME, "or a", BRMSFIT_MODEL_CLASS_NAME, ". Please recheck your model object."))
                               }
                             },
                             
                             #' Sample \eqn{\tilde{\theta} \sim P(\theta)} and \eqn{\tilde{y} \sim P(\tilde{y}) | \tilde{\theta} using \code{sim_function}
                             #'
                             #' @param n_sbc_iterations Number of prior sample sets to generate. Equates to number of SBC iterations
                             #' @param ... Additional arguments to be passed to \code{sim_function}
                             simulate = function(n_sbc_iterations, theta_hp = NULL, ...){  # number of simulation draws should be at least 1000 # how to get D?
                               if(!is.null(self$brms_model_fit)){
                                 #prior_sampler <- update(self$model_obj, sample_prior="only", chains=1, iter=n * 2, warmup = n)
                               }
                               else if(!is.null(self$cmdstan_model)){
                                 draws_rvars_list <- do.call(generator_to_draws_rvars, list(self$sim_function, n_sbc_iterations, theta_hp, ...))
                                 self$prior_samples <- draws_rvars_list[["parameters"]] # receive from generator
                                 self$simulated_y <- draws_rvars_list[["generated"]]
                               }
                               else{
                                 stop("No valid model specified for the workflow. Please check that your model is a valid brmsfit or a CmdStanModel object.")
                               }
                             },
                             
                             #' Sample \eqn{\widehat{\theta} \sim P(\widehat{\theta} | \tilde{y})} using the stan model.
                             #'
                             #' @param sample_iterations Number of sampling iterations. Equates to number of posterior samples
                             #' @param warmup_iterations Number of warmup iteratioins.
                             #' @param data List specifying data to be passed to the model. Note that \code{y} will be replaced with simulated \code{y} samples.
                             #' @param thin_factor Non-negative integer in which thinning should be applied. 1 equates to no thinning being done.
                             #'
                             #' @return A list of named lists containing posterior samples for each parameter.
                             fit_model = function(sample_iterations, warmup_iterations, data=list(), thin_factor=3){
                               self$thin_factor <- thin_factor
                               if(is.null(self$simulated_y)){
                                 stop("There are no simulated data available. Please run SBCWorkflow$simulate() first ")
                               }
                               # if(posterior::nchains(self$prior_samples) != posterior::nchains(self$simulated_y)){
                               #   stop("The number of simulated prior and data do not match. Please rerun SBCWorkflow$simulate() and if using CmdStan, make sure your generator is well formed.")
                               # }
                               
                               
                               if(!is.null(self$brms_model_fit)){
                                 #brms
                               }
                               else if(!is.null(self$cmdstan_model)){
                                 sbc_iterations <- posterior::nchains(self$prior_samples)
                                 posterior_draws_rvars <- cmdstan_fits_to_draws_rvars(self$cmdstan_model, sbc_iterations, data, self$simulated_y, iter_sampling=sample_iterations * thin_factor, iter_warmup = warmup_iterations, chains=1, thin=thin_factor)
                                 self$posterior_samples <- posterior_draws_rvars
                               }
                               
                               posterior_draws_rvars
                             },
                             
                             #' Calculate rank statistics for a given parameter name
                             #'
                             #' @param param list of parameter names to calculate. If not given, calculate for all available parameters.
                             #'
                             calculate_rank = function(param=NULL){
                               if(is.null(self$posterior_samples)){
                                 stop("There are no posterior samples available. Please run SBCWorkflow$fit_model() first.")
                               }
                               if(is.null(param)){
                                 param <- posterior::variables(prior)
                               }
                               calculate_rank_draws_rvars(self$prior_samples, self$posterior_samples)
                               
                               # ranks <- array(rep(0, n_iters * n_params), dim=c(n_iters, n_params))
                               # dimnames(ranks)[2] <- list(names(decomposed_prior))
                               #
                               # for(i in 1:n_iters){
                               #   prior <- self$prior_samples[[i]]
                               #   decomposed_prior <- decompose_structure_to_par_list(prior)
                               #
                               #   for(j in 1:n_params){
                               #     regex_res <- gregexpr("(?<=\\[)(.+)(?=\\])", param[[j]], perl = TRUE)[[1]]
                               #     capture_start <- attr(regex_res, "capture.start")
                               #     capture_length <- attr(regex_res, "capture.length")
                               #     if(capture_start != -1){
                               #       index <- as.integer(unlist(substr(param[[j]], capture_start, capture_start + capture_length - 1)))
                               #       ranks[i, param[[j]]] <- sum(self$posterior_samples[[i]][[param[[j]]]] < access_element_by_index(self$prior_samples[[i]][[base_param_names[[j]]]], index))
                               #     }
                               #     else{
                               #       t1 <- self$posterior_samples[[i]][[param[[j]]]]
                               #       t2 <- self$prior_samples[[i]][[param[[j]]]]
                               #       #ranks[i, param[[j]]] <- sum(self$posterior_samples[[i]][[param[[j]]]] < self$prior_samples[[i]][[param[[j]]]])
                               #       ranks[i, param[[j]]] <- sum(t1 < t2)
                               #     }
                               #   }
                               # }
                               # return(ranks)
                             },
                             add_steps = function(N){
                               
                             }
                           )
)
#utils.R
generator_to_draws_rvars <- function(generator, n_sbc_iterations, theta_hp = NULL, ...){
  merged_parameter_array <- NULL # a 3d array with dimensions(1, n_sbc_iterations, n_variables)
  merged_generated_array <- NULL # a 3d array with dim(n_generated_samples, n_sbc_iterations, 1)
  # we extend the array along the 2nd dimension, which is along chains

  # If there's an easier way to merge rvars along chains, then the double conversion wouldn't be needed
  for(iter in 1:n_sbc_iterations){
    generator_output <- do.call(generator, theta_hp = theta_hp, list(...))
    parameter_array <- posterior::as_draws_array(posterior::as_draws_rvars(generator_output[["parameters"]]))  # No clean way to directly transform generator output to draws_array
    generated_array <- posterior::as_draws_array(posterior::as_draws_rvars(list(y=posterior::rvar(generator_output[["generated"]]))))
    dimnames(parameter_array)[[2]] <- iter
    dimnames(generated_array)[[2]] <- iter
    if(iter == 1){
      merged_parameter_array <- parameter_array
      merged_generated_array <- generated_array
    }
    else{
      merged_parameter_array <- abind::abind(merged_parameter_array, parameter_array, along=2)
      merged_generated_array <- abind::abind(merged_generated_array, generated_array, along=2)
    }
  }
  list(parameters=posterior::as_draws_rvars(merged_parameter_array), generated=posterior::as_draws_rvars(merged_generated_array))
}
cmdstan_fits_to_draws_rvars <- function(cmdstan_model, n_sbc_iterations, data_list, simulated_y_draws_rvars, ...){
  if(n_sbc_iterations != posterior::nchains(simulated_y_draws_rvars)){
    stop("Number of chains in simulated_y_draws_rvars does not match n_sbc_iterations")
  }
  merged_fits_array <-NULL # a 3d array with dimensions(n_posterior_samples, n_sbc_iterations, n_variables)

  for(iter in 1:n_sbc_iterations){
    data_list[["y"]] <- as.vector(posterior::draws_of(posterior::subset_draws(simulated_y_draws_rvars, chain=iter)$y))
    model_fit <-do.call(cmdstan_model$sample, list(data=data_list, ...))
    fit_array <- posterior::as_draws_array(posterior::as_draws_rvars(model_fit$draws()))
    dimnames(fit_array)[[2]] <- iter

    if(iter == 1){
      merged_fits_array <- fit_array
    }
    else{
      merged_fits_array <- abind::abind(merged_fits_array, fit_array, along=2)
    }
  }
  posterior::as_draws_rvars(merged_fits_array)
}
calculate_rank_draws_rvars <- function(prior_draws_rvars, posterior_draws_rvars){
  n_vars <- posterior::nvariables(prior)
  sbc_iters <- posterior::nchains(prior)
  merged_rank_array <- NULL
  for(n_iter in 1:sbc_iters){
    rank_list <- list()
    for(n_var in 1:n_vars){
      var <- posterior::variables(prior)[[n_var]]

      prior_slice <- as.array(posterior::draws_of(posterior::subset_draws(prior, variable=var, chain=n_iter)[[var]]))
      iter_comp <- apply(posterior::draws_of(posterior::subset_draws(post, variable = var, chain=n_iter)[[var]]), 1, function(x){x < prior_slice})
      if(is.vector(iter_comp)){
        rank_list[[var]] <- sum(iter_comp, na.rm=TRUE)
      }
      else{
        rank_slice <- apply(iter_comp, c(1:(max(1,length(dim(iter_comp))-1))), function(x){sum(x, na.rm=TRUE)})
        # Need to test for scalar and matrices
        rank_list[[var]] <- rank_slice
      }

    }
    rank_array <- posterior::as_draws_array(posterior::as_draws_rvars(rank_list))
    dimnames(rank_array)[[2]] <- n_iter
    if(n_iter == 1){
      merged_rank_array <- rank_array
    }
    else{
      merged_rank_array <- abind::abind(merged_rank_array, rank_array, along=2)
    }
  }
  posterior::as_draws_rvars(merged_rank_array)
}

In [111]:
generator_output <- do.call(generator, list(...))

ERROR: Error in do.call(generator, list(...)): '...' used in an incorrect context


In [1]:
library(cmdstanr)
#install_cmdstanr(overwrite = TRUE)
library(dplyr)
library(tidyverse)
library(reshape)
library(parallel)
library(posterior)
#library(rstanarm)
devtools::install_github("hyunjimoon/SBC",ref="workflow")
library(SBC)
library(pracma)
library("entropy")
#devtools::install_github("rmcelreath/rethinking",ref="Experimental")
#library(rethinking)
set.seed(1954)
.libPaths("~/Rlib")
options(mc.cores = parallel::detectCores())
set_get_Dir <- function(modelName){
  scriptDir <- getwd()
  modDir <- file.path(scriptDir, "models")
  dataDir <- file.path(scriptDir, "data")
  delivDir <- file.path(scriptDir, "deliv", modelName)
  dir.create(delivDir)
  file <- file.path(modDir, modelName, paste0(modelName, ".stan"))
  mod <- cmdstan_model(file)
  return(list(data = data, mod = mod, modDir = modDir, file = file, delivDir = delivDir)) 
}

source("tools/selfCalib.R")
modelName = "simple.normal_1"
modDir <- set_get_Dir(modelName)$modDir
delivDir <- set_get_Dir(modelName)$delivDir
file <- set_get_Dir(modelName)$file
ncp_model = cmdstanr::cmdstan_model(file)
generator <- function(){
  function(){
    mu <- rnorm(1, 0, 5)
    tau <- rcauchy(1, 0, 5)
    theta_trans <- rnorm(8, 0, 1)
    theta <-theta_trans * tau + mu
    list(
      generated = rnorm(8, mu, sigma),
      parameters = list(
        mu = mu,
        theta_trans=theta_trans,
        tau = tau,
        theta = theta
      )
    )
  }
}

J <- 8
y <- c(28, 8, -3, 7, -1, 1, 18, 12)
sigma <- c(15, 10, 16, 11, 9, 11, 10, 18)
nChains <- 4
parallel_chains <- min(nChains, detectCores())
N = 100 #200
M = 20 # 40


This is cmdstanr version 0.4.0.9000

- Online documentation and vignettes at mc-stan.org/cmdstanr

- CmdStan path set to: /Users/hyunjimoon/.cmdstan/cmdstan-2.27.0

- Use set_cmdstan_path() to change the path


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ stringr 1.4.0
✔ tidyr   1.1.3     ✔ forcats 0.5.1
✔ readr   1.4.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape’


The following objects are masked from ‘package:tidyr’:

    expand, smiths


The following object is masked from ‘package:dplyr’:

    rename


This is posterior version 1.0.0


Attachin